## ChatGPT Investment Advice

#### This script automates investment advice using the ChatGPT API.

Using the Python script together with the ChatGPT API allows this chatbot to be used on an commercial scale. I.e., it fully automates investment advice and can easy be integrated into a webpage or other such user interface.

The chatbot replicates front-of-house investment advice that you might recieve at a commerial bank.  

In [1]:
import json
import os
from collections import OrderedDict
from datetime import datetime
from pathlib import Path

from openai import OpenAI   # pip install openai>=1.0.0
from openai_key import OPENAI_API_KEY


#### Step 1: Define the questions

In [10]:
QUESTIONS = OrderedDict({
    "objectives_time_horizon":
        ("1. Investment Objectives & Horizon:\n"
         "   What are your specific financial goals (retirement, property "
         "acquisition, education funding), and what is your anticipated timeline for "
         "accessing these funds?\n> "),

    "risk_tolerance_capacity":
        ("2. Risk Tolerance Assessment:\n"
         "   What level of portfolio volatility are you comfortable with, and what "
         "maximum temporary decline (percentage or monetary value) could you tolerate "
         "without altering your investment strategy?\n> "),

    "current_financial_position":
        ("3. Financial Position Overview:\n"
         "   Please summarize your current income, major assets, liabilities, and "
         "emergency reserves to determine your optimal investment capacity.\n> "),

    "liquidity_cash_flow_needs":
        ("4. Liquidity Requirements:\n"
         "   Do you anticipate significant expenditures (property down payment, "
         "healthcare costs, career transition) within the next 1-5 years that would "
         "necessitate liquidity from this portfolio?\n> "),

    "constraints_preferences_tax":
        ("5. Investment Parameters:\n"
         "   Please identify any relevant investment restrictions, ESG preferences,  "
         "tax considerations, or jurisdictional factors that should guide your "
         "portfolio construction.\n> "),
})

#### Step 2. Collect responses from the user

In [11]:
def ask_questions() -> OrderedDict:
    print("\n--- Investor Profile Questionnaire ---\n")
    answers = OrderedDict()
    for key, prompt in QUESTIONS.items():
        answers[key] = input(prompt).strip()
        print("\n")
    return answers

In [12]:
def save_json(data: OrderedDict, folder: str = "client_profiles") -> Path:
    Path(folder).mkdir(exist_ok=True)
    filename = Path(folder) / f"profile_{datetime.now():%Y%m%d_%H%M%S}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    return filename

In [13]:
answers = ask_questions()


--- Investor Profile Questionnaire ---

1. Investment Objectives & Horizon:
   What are your specific financial goals (retirement, property acquisition, education funding), and what is your anticipated timeline for accessing these funds?
> Retirement in 40 years.


2. Risk Tolerance Assessment:
   What level of portfolio volatility are you comfortable with, and what maximum temporary decline (percentage or monetary value) could you tolerate without altering your investment strategy?
> I have a high risk tolerance and would not change strategy until there is a 10% decline in portfolio assets.


3. Financial Position Overview:
   Please summarize your current income, major assets, liabilities, and emergency reserves to determine your optimal investment capacity.
>  Next year I hope to start earning 70K annually, which should gradually increase. I have no major assets or liabilities, and I have 10K in cash.


4. Liquidity Requirements:
   Do you anticipate significant expenditures (prope

In [14]:
save_path = save_json(answers)
print(f"\nResponses saved to {save_path.resolve()}\n")


Responses saved to /content/client_profiles/profile_20250428_033125.json



#### Step 3. Build the prompt & call ChatGPT

In [15]:
SYSTEM_PROMPT = (
    "Forget all your previous instructions. Pretend you are an expert financial advisor. "
    "You are an expert financial advisor with investment and portfolio allocation experience."
)

def call_chatgpt(q_and_a: OrderedDict, model: str = "gpt-4o-mini") -> str:
    """Send the Q&A context plus the instruction to ChatGPT and return the reply."""
    # Convert Q&A to a readable block
    qa_block = "\n".join(
        f"{idx}. Q: {QUESTIONS[key].split(':')[0]}  A: {answer}"
        for idx, (key, answer) in enumerate(q_and_a.items(), start=1)
    )

    user_message = (
        f"Your client has responsed to five onboarding questions as follows:\n{qa_block}\n\n"
        "Given the situation described by the responses to the five investment questions, "
        "what investment strategy should be followed? Provide two clear and concise sentences "
        "that explain this choice of strategy without financial jargon."
        "Subsequently, identify three suitable funds."
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_message},
        ],
        temperature=0  # deterministic
    )

    return response.choices[0].message.content.strip()

In [16]:
client = OpenAI(api_key=OPENAI_API_KEY)

strategy = call_chatgpt(answers)

In [17]:
print(strategy)

Given your long investment horizon and high risk tolerance, a growth-oriented strategy focusing on equities is appropriate. This approach aims to maximize returns over time, taking advantage of market fluctuations while you have the ability to ride out volatility.

Three suitable funds for this strategy could be:
1. Vanguard Total Stock Market Index Fund (VTSAX) - provides broad exposure to the U.S. stock market.
2. Fidelity Growth Company Fund (FDGRX) - focuses on high-growth companies with strong potential.
3. T. Rowe Price Global Technology Fund (PRGTX) - invests in technology companies worldwide, capitalizing on innovation and growth trends.
